In [ ]:
import pandas as pd
from pandas import DataFrame, Series

import os

import warnings
warnings.filterwarnings('ignore')

import time
import timeit

import joblib
#import ENG
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models.coherencemodel import CoherenceModel
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

import pythainlp
from pythainlp import word_tokenize

import re

def splitword(text):
  out=[]
  couUp=0
  left=0
  initial=text.strip()
  upper=re.findall('[A-Z]',initial)
  for i in range(len(initial)):
    if initial[i] in upper:
      couUp+=1
    if couUp==len(upper):
      out.append(initial[left:i])
      out.append(initial[i:])
      break
    elif initial[i] in upper and i !=0:
      out.append(initial[left:i])
      left=i
  return ' '.join(out).strip()

#Data Pre-processing for English only
# input = many words
#output = [w1, w2, w3, ......]

#1. Tokenization: Split the text into sentences and the sentences into words.
# Lowercase the words and remove punctuation.
#2. Words that have fewer than 3 characters are removed.
#3. All stopwords are removed.
#4. Words are lemmatized — 
#words in third person are changed to first person and verbs in past and future tenses are changed into present.
#5. Words are stemmed — words are reduced to their root form.
def lemmatize_stemming(text):
    return SnowballStemmer('english').stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

#process for all data
#input = docs
# output = a list of each word
def preprocessAll(text):
  text=str(text)
  text=text.strip()
  thai=pythainlp.thai_characters
  th=''
  en=''
  for i in range(len(text)):
    if text[i] in thai:
      th=th+text[i]
    else:
      if (i != 0) and (text[i-1] in thai):
        en=en+' '+text[i]
      else:
        en=en+text[i]
  temp_en=[]
  for i in en.split(' '):
    if i !='':
      temp_en.append(splitword(i))
  temp_en=' '.join(temp_en)
  return preprocess(temp_en)+word_tokenize(th, keep_whitespace=False)

def predict_tfid_loadedModel(text, loaded_model):
  process_unseen= preprocessAll(text)
  dic_unseen=loaded_model.id2word
  bow_unseen=dic_unseen.doc2bow(process_unseen) 
  target=bow_unseen
  return sorted(loaded_model[target], key= lambda x:x[1], reverse= True)

def result(text, rec):
    out=[]
    con=1
    for item in rec:
        t= predict_tfid_loadedModel(text, item[0])
        if len(t) >0:
            if str(t[0][0]) in item[2]:
                out.append([item[1], t[0][1]])
                con=0
    if con==1:
        out= 'Others'
    return out
start = timeit.default_timer()
''' # load selected model '''
select=pd.read_csv('/home/std/Downloads/csv_file/model_selection_orderTii_V2.csv')
rec=[]
model_path='/home/std/Downloads/python/model_temp/Final_title_noBiTri_varyTopic_dictionaryV3_AlphaEtaV1/'
#all_model=os.listdir(model_path)
for num in sorted(list(select['order'].unique())):
    #print(num)
    job=select[select['order']==num]['job'].iloc[0]
    #print(job)
    alp=select[select['order']==num]['alpha'].iloc[0]
    #print(alp)
    beta=select[select['order']==num]['beta'].iloc[0]
    #print(beta)
    n_topic=select[select['order']==num]['n_topic'].iloc[0]
    #print(n_topic)
    rep=re.findall('[0-9]{1,3}', select[select['order']==num]['Represent_topic'].iloc[0])
    #print(rep)
    model=joblib.load(model_path+'[below:topic:alpha:eta]_15:'+str(n_topic)+':'+str(alp)+':'+str(beta)+'.sav')
    rec.append([model, job, rep])
    

    
test=pd.read_csv('/home/std/Downloads/csv_file/test_700kminus6261_combine.csv')
print('len test', len(test))
select2=test['combine']
# keep=[]
# for i in range(len(select)):
#     keep.append(result(select.iloc[i], rec))
# test['result']=pd.Series(keep, index=test.index)



[nltk_data] Downloading package wordnet to /home/std/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


len test 714849


In [ ]:
def deploy_model(tar):
    start = timeit.default_timer()
    cou=0
    print('$$$$$$ '+tar+'\n')
    cal=0
    for i in range(len(test)):
        temp_start = timeit.default_timer()
        text=select2[i]
        if result(text, rec) != 'Others':
            lol=[h[0] for h in result(text, rec)]
            #print(lol)
            if tar in lol:
                print(text)
                print('#############################################################')
                print(' ')
                cou+=1
                if cou==10:
                    break
        temp_stop = timeit.default_timer()
        cal+= temp_stop - temp_start
        if cal>= 600:
            print('time out (more than 10 mins)')
            break

        # print(predict_tfid_loadedModel(text, rec[0][0]))
        # print(rec[0])
        #print(result(text, rec))

    stop = timeit.default_timer()
    print('runtime: ', stop - start)
    print('\n'+'\n')

In [ ]:
jobs=['coordinator',
 'Investor // finance // account',
 'Others',
 'marketing // advertising',
 'Policy manager // BD',
 'Stock // procurement // supply',
 'teacher',
 'law',
 'cooker + bartender',
 'translator',
 'HR',
 'sale // service',
 'manufacturing manager',
 'writer',
 'security + safety',
 'credit analyst',
 'graphic design',
 'retail manager',
 'maid',
 'therapist',
 'waiter',
 'architecture',
 'vehicle driver',
 'care taker',
 'cashier']

In [ ]:
for job in jobs:
    deploy_model(job)

$$$$$$ coordinator

 พนักงานสินค้า/ส่งเอกสารประจำ อำเภอเมือง จังหวัดสกลนคร ด่วน!!!! Responsibilitiesส่งเอกสารที่ได้รับหมอบหมายให้สำเร็จส่งเอกสารให้ถึงมือลูกค้าโดยถูกต้องQualificationsเพศชาย อายุ 18-35 ปี วุฒิการศึกษา ม.3 - ปริญญาตรี มีที่อยู่อาศัยในอำเภอเมือง จังหวัดสกลนคร สามารถปฏิบัติงาน ประจำที่ศูนย์กิฟฟารีน จ.สกลนครได้มีใบอนุญาติขับขี่รถจักรยานยนต์ มีรถจักรยานยนต์เป็นของตัวเอง มีความขยัน อดทน ซื่อสัตย์ต่อองค์กร เริ่มงานได้ทันทีจะพจารณาเป็นพิเศษ ทำงาน จ-ส เวลา09.00-18.00 Benefitsมีเบี้ยขยัน และสวัสดิการตามประกันสังคมมีความมั่นคงในตำแหน่งหน้าที่ ContactProntima21 ซอยรามคำแหง 21(ใกล้เซเว่นกลางซอย) ถนนรามคำแหง แขวงพลับพลา เขตวังทองหลาง กรุงเทพมหานคร 10310Tel : 02-718-5100
#############################################################
 
 เลขานุการผู้บริหาร รายละเอียดงานดูแลตารางการนัดหมายของผู้บริหาร ทั้งภายใน และภายนอกติดตามงาน แจ้งงานรายละเอียด- ความคืบหน้า ทำรายงาน ต่อผู้บริหารติดต่อ-ประสานงาน ทั้งภายในและภายนอกจัดทำนัดหมายการประชุม ทำรายงานการประชุม ติดตามผลหลังการประชุมงานอื่นๆ ตามท

In [ ]:
text="เลขานุการ 1.ประสานงาน,จัดตารางนัดหมาย,เตรียมเอกสาร/บันทึกการประชุม 2.ติดต่อกับบุคคล/หน่วยงานต่างๆ ทั้งในและต่างประเทศ3.งานอื่นๆ ที่ได้รับมอบหมายจากผู้บริหาร"
result(text, rec)

[['care taker', 0.37605983], ['coordinator', 0.67410403]]

In [ ]:
result('love you', rec)

'Others'

In [ ]:
result('finance', rec)

[('Investor // finance // account', 0.8867252)]

In [ ]:
result('master degree Engineering', rec)

[('interior archetecture', 0.19089705), ('Policy manager // BD', 0.10914666)]

In [ ]:
result('finance stock procurement sale marketing advertising manager recruit', rec)

[('marketing // advertising', 0.2540577)]